In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')

DIRECTORY = '/content/drive/Shareddrives/CMPE295/Food_Com_Dataset/finetuned/ingrs' 

if os.getcwd() != DIRECTORY:
  os.chdir(DIRECTORY)

Mounted at /content/drive


## Food.Com Dataset

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('./database.csv')

In [3]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
df.sample(5)

,name,minutes,contributor_id,submitted,tags,n_steps,description,n_ingredients,cal,totalFat,sugar,sodium,protein,satFat,carbs,image,ingredients,steps,id,servings
30858,cajun alfredo sauce,30,358681,2008-04-20,"['keto', 'sauces', 'savory', 'sauce', 'north-a...",8,a simple yet delicious alfredo sauce. you can...,5,548.7,89.0,0.0,10.0,11.0,180.0,1.0,"https://img.sndimg.com/food/image/upload/q_92,...","['1⁄2 cup butter', '1 quart heavy cream', '1 t...",Melt butter.\nAdd garlic.\nSaute till you smel...,299505,8
148321,paula deen s breakfast strata,615,175492,2007-07-23,"['breakfast', '3-steps-or-less', 'eggs', 'dair...",14,i found this recipe in paula deen's magazine. ...,15,834.7,86.0,21.0,103.0,87.0,161.0,13.0,"https://img.sndimg.com/food/image/upload/q_92,...","['3 tablespoons butter', '1 small onion, chopp...",Lightly grease a 13x9x2-inch baking dish.\nIn ...,242115,6-8
71282,easy and flavourful canned tomatoes,123,130080,2005-08-23,"['canning', 'tomatoes', 'soups', 'stews', 'veg...",14,recipe zaar is a great site for inspiration. ...,8,187.5,3.0,80.0,1.0,17.0,2.0,14.0,"https://img.sndimg.com/food/image/upload/q_92,...","['20 lbs roma tomatoes', '1⁄4 cup rock salt', ...",You will need 12 one litre jars or 1.5 litre j...,134658,NaN
85163,fruit salad with custard,37,6357,2002-09-21,"['asian', '60-minutes-or-less', 'dessert', 'in...",26,this is a special dessert from today's thursda...,14,569.3,28.0,195.0,13.0,28.0,60.0,30.0,"https://img.sndimg.com/food/image/upload/q_92,...","['For the fruit salad', '3 cups mixed fruit, f...",For the fruit salad: Combine all the ingredien...,40889,6
76527,elegant sunday morning eggs,25,118930,2004-02-21,"['breakfast', '30-minutes-or-less', 'for-1-or-...",8,the following recipe is for one serving. doubl...,6,306.7,25.0,6.0,39.0,43.0,37.0,5.0,"https://img.sndimg.com/food/image/upload/q_92,...","['2 slices black forest ham, thin not shaved',...",Preheat oven to 350°F.\nSpray a Pyrex custard ...,84660,1


In [6]:
df[['name', 'tags']].sample(5)

,name,tags
15997,bananas foster oatmeal,"['breakfast', 'vegan', 'vegetarian', 'for-1-or..."
178076,shrimp pasta salad with asian dressing,"['asian', 'vegetables', 'shellfish', 'salads',..."
110921,john s bread salad,"['vegetables', 'main-dish', 'salads', 'one-dis..."
222191,zucchini cheese patties,"['vegetables', '30-minutes-or-less', 'side-dis..."
82668,french onion grilled cheese from the sandwich...,"['lunch', 'american', 'eggs', 'dairy', 'north-..."


In [ ]:
import ast

def eval_str(s):
  lst = list(set(ast.literal_eval(s)))
  return ' '.join(lst) + '\n'

In [ ]:
from tqdm.auto import tqdm

with tqdm(total = df.shape[0]) as pbar, open('./ingrs.txt', 'a') as f:
  for _, ingr in df['ingredients'].iteritems():
    ingr = eval_str(ingr)
    f.write(ingr)
    pbar.update(1)

In [ ]:
from tqdm.auto import tqdm

train = df['ingredients'].sample(frac=0.8, replace=False)
training_set = set(train.unique())
test = []

for _, ingr in df['ingredients'].iteritems():
  if ingr not in training_set:
    test.append(ingr)

test = pd.Series(test)

with tqdm(total = train.shape[0]) as pbar, open('./train.txt', 'a') as f:
  for _, ingr in train.iteritems():
    ingr = eval_str(ingr)
    f.write(ingr)
    pbar.update(1)

with tqdm(total = test.shape[0]) as pbar, open('./test.txt', 'a') as f:
  for _, ingr in test.iteritems():
    ingr = eval_str(ingr)
    f.write(ingr)
    pbar.update(1)

  0%|          | 0/178359 [00:00<?, ?it/s]

  0%|          | 0/44212 [00:00<?, ?it/s]

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers
!pip list | grep -E 'transformers|tokenizers'

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)
sentence-transformers         2.1.0
tokenizers                    0.10.3
transformers                  4.12.3


In [ ]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = ['./ingrs.txt']
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 27.1 s, sys: 4.23 s, total: 31.3 s
Wall time: 1.23 s


In [ ]:
!mkdir ingrsBERT
tokenizer.save_model("ingrsBERT")

['ingrsBERT/vocab.json', 'ingrsBERT/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./ingrsBERT/vocab.json",
    "./ingrsBERT/merges.txt",
)

In [ ]:
tokenizer.enable_truncation(max_length=512)

In [ ]:
from transformers import RobertaConfig
from transformers import RobertaTokenizerFast
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling

tokenizer = RobertaTokenizerFast.from_pretrained("./ingrsBERT", max_len=512)
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
train = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./train.txt",
    block_size=64,
)
test = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./test.txt",
    block_size=64,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

file ./ingrsBERT/config.json not found
file ./ingrsBERT/config.json not found
/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./ingrsBERT",
    num_train_epochs=10,
    per_gpu_train_batch_size=64,
    learning_rate=2e-5,
    save_steps=10_000,
    save_total_limit=2,
    weight_decay=0.01,
    prediction_loss_only=True,
    load_best_model_at_end=True,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train,
    eval_dataset=test,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('./ingrsBERT')

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 3.1 MB 11.1 MB/s 
     |████████████████████████████████| 3.3 MB 70.2 MB/s 
     |████████████████████████████████| 1.2 MB 56.7 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 895 kB 54.8 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=f6303dcb3efea7c0fb5f580a4c2ee12e7d6fc0e5ecd9ce88742ba6a3f0a9b494
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('./ingrsBERT')
sbert_model = SentenceTransformer('all-mpnet-base-v2')

Some weights of the model checkpoint at ./ingrsBERT were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./ingrsBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
s1 = model.encode('garlic pasta pork_belly kimchi')
s2 = model.encode('garlic rice pork_belly gochujang')

In [ ]:
t1 = sbert_model.encode('garlic pasta pork_belly kimchi')
t2 = sbert_model.encode('garlic rice pork_belly gochujang')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity([s1, t1], [s2, t2])

array([[0.830755  , 0.0363441 ],
       [0.01291469, 0.76876694]], dtype=float32)

In [ ]:
from tqdm.auto import tqdm
import numpy as np

with tqdm(total=df.shape[0]) as pbar, open('./embeddings.csv', 'a') as f:
  for _, row in df.iterrows():
    ingr = eval_str(row['ingredients'])
    ingr_enc = model.encode(ingr)
    tag = eval_str(row['tags'])
    tag_enc = sbert_model.encode(tag)
    embedding = np.hstack((tag_enc, ingr_enc))
    np.savetxt(f, [embedding], delimiter=',')
    pbar.update(1)

  0%|          | 0/222949 [00:00<?, ?it/s]

## Recipe 1M Dataset

In [ ]:
df = pd.read_json('../../../Recipe1M_Enriched_Dataset/recipe1M.json')

In [ ]:
df['tags_unique'].sample(1).iloc[0]

['desserts', 'dessert']

In [ ]:
df.columns

Index(['id', 'title', 'url', 'instructions_text', 'ingredients_layer1',
       'images_url', 'tags_unique', 'short_ingredients'],
      dtype='object')

In [ ]:
sbert_model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df.shape

(1029718, 8)

In [ ]:
from tqdm.auto import tqdm
import numpy as np

with tqdm(total=df.shape[0]) as pbar, open('./embeddings1M.csv', 'a') as f:
  for _, row in df.iterrows():
    ingr = eval_str(row['short_ingredients'])
    ingr_enc = model.encode(ingr)
    tag = eval_str(row['tags_unique'])
    tag_enc = sbert_model.encode(tag)
    embedding = np.hstack((tag_enc, ingr_enc))
    np.savetxt(f, [embedding], delimiter=',')
    pbar.update(1)

# Weaviate

## Upload Embeddings

In [ ]:
import pandas as pd
import numpy as np

database = pd.read_csv('./database.csv')

In [ ]:
database.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [ ]:
def clean_df(df: pd.DataFrame):
  df.fillna(value = {
      'name': '',
      'minutes': -1,
      'contributor_id': -1,
      'submitted': '',
      'tags': '',
      'n_steps': -1,
      'description': '',
      'n_ingredients': -1,
      'cal': -1,
      'totalFat': -1.0,
      'sugar': -1.0,
      'sodium': -1.0,     
      'protein': -1.0,   
      'satFat': -1.0,    
      'carbs': -1.0,     
      'image': '',     
      'ingredients': '',
      'steps': '',     
      'id': -1,        
      'servings': '', 
      },
      inplace=True
  )

In [ ]:
clean_df(database)

In [ ]:
!pip install weaviate-client

     |████████████████████████████████| 59 kB 3.6 MB/s 


In [ ]:
import weaviate
import pandas as pd 
import numpy as np 
import ast
import json
import pprint
from tqdm.auto import tqdm
from weaviate.batch import Batch

pp = pprint.PrettyPrinter(indent=4)

def init_weaviate_schema():
  return {
    "classes": [{
      "class": "Recipe",
      "vectorizer": "none",
      "properties": [
        {
          "name": "rid",
          "dataType": ["string"],
        },
        {
          "name": "name",
          "dataType": ["string"],
        },
        {
          "name": "minutes",
          "dataType": ["int"],
        }, 
        {
          "name": "tags",
          "dataType": ["string[]"],
        }, 
        {
          "name": "description",
          "dataType": ["string"],
        }, 
        {
          "name": "cal",
          "dataType": ["number"],
        }, 
        {
          "name": "totalFat",
          "dataType": ["number"],
        }, 
        {
          "name": "sugar",
          "dataType": ["number"],
        }, 
        {
          "name": "sodium",
          "dataType": ["number"],
        }, 
        {
          "name": "protein",
          "dataType": ["number"],
        }, 
        {
          "name": "satFat",
          "dataType": ["number"],
        }, 
        {
          "name": "carbs",
          "dataType": ["number"],
        }, 
        {
          "name": "image",
          "dataType": ["string"],
        }, 
        {
          "name": "ingredients",
          "dataType": ["string[]"],
        },
        {
          "name": "model_ingredients",
          "dataType": ["string[]"],
        }, 
        {
          "name": "steps",
          "dataType": ["string"],
        }, 
        {
          "name": "servings",
          "dataType": ["string"]
        }
        ]
    }]
  }

def init_weaviate_client():
  my_credentials = weaviate.auth.AuthClientPassword('naruto', 'SRRE295b')
  return weaviate.Client('http://20.120.216.46:8080/', auth_client_secret=my_credentials)
  
def create_schema():
  client = init_weaviate_client()
  client.schema.create(init_weaviate_schema())

def create_recipe_object(row):
  return {
    "rid": str(row['id']),
    "name": row['name'],
    "minutes": row['minutes'],
    "tags": ast.literal_eval(row['tags']),
    "description": row['description'],
    "cal": row['cal'],
    "totalFat": row['totalFat'],
    "sugar": row['sugar'],
    "sodium": row['sodium'],
    "protein": row['protein'],
    "satFat": row['satFat'],
    "carbs": row['carbs'],
    "image": row['image'],
    "ingredients": ast.literal_eval(row['ingredients']),
    "model_ingredients": ast.literal_eval(row['model_ingredients']),
    "steps": row['steps'],
    "servings": row['servings']
  }

def add_recipe(batch: Batch, recipe_object: dict, vector):
  batch.add_data_object(
      data_object=recipe_object,
      class_name='Recipe',
      vector=vector
  )


In [ ]:
type(init_weaviate_client())

weaviate.client.Client

In [ ]:
def import_recipes_with_vectors():
  client = init_weaviate_client()
  batch_size = 16
  with tqdm(total=database.shape[0]) as pbar, open('./err.txt', 'a+') as err, open('./logs.txt', 'a+') as logs:    
    count = 0
    num_rows = 0

    for idx, row in database.iterrows():
      if idx % 1024 == 0:
        arr = np.loadtxt('./embeddings.csv', delimiter=',', skiprows=count*1024, max_rows=1024)
        count += 1
      try:
        recipe_object = create_recipe_object(row)
      except TypeError:
        err.write(str(idx) + ' ' + str(row['id'])) 
        continue

      add_recipe(client.batch, recipe_object, arr[idx%1024])
      num_rows += 1
      if num_rows % batch_size == 0:
        client.batch.create_objects()
        pbar.update(batch_size)


In [ ]:
my_credentials = weaviate.auth.AuthClientPassword('naruto', 'SRRE295b')
client = weaviate.Client('http://20.120.216.46:8080/', auth_client_secret=my_credentials)
client.schema.delete_class(class_name='Recipe')
create_schema()
import_recipes_with_vectors()

  0%|          | 0/222949 [00:00<?, ?it/s]

In [ ]:
# my_credentials = weaviate.auth.AuthClientPassword('naruto', 'SRRE295b')
# client = weaviate.Client('http://20.120.216.46:8080/', auth_client_secret=my_credentials)
# client.schema.delete_class(class_name='Recipe')
# create_schema()

## Search

In [ ]:
!pip install sentence-transformers
!pip install transformers

     |████████████████████████████████| 78 kB 3.6 MB/s 
     |████████████████████████████████| 3.1 MB 19.5 MB/s 
     |████████████████████████████████| 3.3 MB 50.4 MB/s 
     |████████████████████████████████| 1.2 MB 65.4 MB/s 
     |████████████████████████████████| 59 kB 8.4 MB/s 
     |████████████████████████████████| 596 kB 79.2 MB/s 
     |████████████████████████████████| 895 kB 85.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=209d817b4f6539fe1d6d1b4a5e2de7692c8b267e0081c9e8502da82ec7a8309e
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('./ingrsBERT')
sbert_model = SentenceTransformer('all-mpnet-base-v2')

Some weights of the model checkpoint at ./ingrsBERT were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./ingrsBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def search(query: dict, limit=5):
  client = init_weaviate_client()
  ingrs_str = ' '.join(query['ingrs']) + '\n'
  tags_str = ' '.join(query['tags']) + '\n'
  ingrs = model.encode(ingrs_str)
  tags = sbert_model.encode(tags_str)
  embedding = np.hstack((tags, ingrs))
  q = {"vector": embedding.tolist()}
  schema = client.schema.get(class_name="Recipe")
  res = client.query.get(class_name="Recipe", properties=[d['name'] for d in schema['properties']] + ["_additional {certainty}"]).with_near_vector(q).with_limit(limit).do()
  return res['data']['Get']['Recipe']

In [ ]:
query = {
    'ingrs': ['chicken', 'mushrooms', 'spinach'],
    'tags': ['low-sodium', 'kid-friendly'],
    'exclusions_ingrs': [''],
    'exclusions_tags': [''],
}

{
  "ingredients": ["chicken", "mushrooms", "spinach"],
  "tags": ["low_sodium", "kid-friendly"],
  "ingredient_exclusions": [],
  "tag_exclusions": [],
  "limit": 10
}

In [ ]:
res = search(query, limit=1000)

In [ ]:
def validate_results(res, query):
  validated = []
  
  for d in res:
    tags = d['tags']
    ingrs = d['model_ingredients']
    intersection_ingrs = list(set(ingrs) & set(query['ingrs']))
    intersection_tags = list(set(tags) & set(query['tags']))
    intersection = (0.667*len(intersection_ingrs)) + (0.333*len(intersection_tags))
    if intersection > 0:      
      validated.append((d, intersection))

  validated.sort(key=lambda x: x[1], reverse=True)
  return validated, len(validated)

In [ ]:
validated, count = validate_results(res, query)

In [ ]:
validated[0]

({'_additional': {'certainty': 0.8631509},
  'cal': 759.6,
  'carbs': 27,
  'description': 'posted for zwt7- the south pacific islands. this yummy samoan recipe was found on the internet.',
  'image': 'https://img.sndimg.com/food/image/upload/q_92,fl_progressive,w_1200,c_scale/v1/img/recipes/45/58/40/picgvcqNm.jpg',
  'ingredients': ['2 lbs chicken, cut into 8 pieces',
   '1 large onion, chopped',
   '1⁄2 lb spinach, cut in 1/4-inch slices or 1/2 lb taro root, leaves cut in 1/4-inch slices',
   '2 lbs taro root, peeled and cut into 1/4-inch slices',
   '6 ounces tomatoes, chopped',
   '2 teaspoons cornflour',
   '1 (14 ounce) can coconut cream',
   '1⁄2 coconut cream can water',
   '1 teaspoon salt',
   'lemon slice'],
  'minutes': 80,
  'model_ingredients': ['chicken',
   'onion',
   'spinach',
   'taro_root',
   'tomatoes',
   'cornflour',
   'coconut_cream',
   'water',
   'salt',
   'lemon_slice'],
  'name': 'taro and chicken coconut cream casserole',
  'protein': 65,
  'rid': '455

In [ ]:
type(model)

sentence_transformers.SentenceTransformer.SentenceTransformer

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = 'roberta-base-squad2-finetuned-selqa-3'
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)